import lib

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

request for url, make a soup

In [2]:
target_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
res = requests.get(target_url)
html_data=res.text
soup = BeautifulSoup(html_data,"html5lib")
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

do some tests

In [23]:
soup.select('tbody td span')[2].text

'North York(Parkwoods)'

attr 1

In [19]:
soup.select('tbody td b')[2].text

'M3A'

attr 2

In [29]:
soup.select('tbody td span')[1].text

'Not assigned'

attr 3

In [28]:
soup.select('tbody td span')[1].text

'Not assigned'

collect info

### Attr 1

In [26]:
attr1=[]
for row in soup.select('tbody td b'):
  attr1.append(row.text)

### Attr 2 & 3

In [34]:
'Not assigned'.replace(')','')

'Not assigned'

In [39]:
attr2=[]
attr3=[]

for row in soup.select('tbody td span'):
  attr2.append(row.text.split('(')[0])
  if len(row.text.split('('))>1:
    attr3.append(row.text.split('(')[1].replace(')','').replace(' /',','))
  else:
    attr3.append(None)

In [40]:
print(len(attr1),len(attr2),len(attr3))

180 180 180


In [41]:
attr3[4]

'Regent Park, Harbourfront'

In [43]:
row_data=list(zip(attr1,attr2,attr3)); row_data[1]

('M2A', 'Not assigned', None)

In [44]:
row_data[1][1]

'Not assigned'

filter the data within 'Not assigned'

In [46]:
filtered_row_data=[x for x in row_data if x[1]!='Not assigned']; len(filtered_row_data)

103

create the dataframe

In [47]:
df=pd.DataFrame(filtered_row_data,columns=['PostalCode','Borough','Neighborhood']); df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [50]:
df[df['Neighborhood'].str.contains('Not')]

,PostalCode,Borough,Neighborhood


In [51]:
df.shape

(103, 3)

Thank you for watching.

In [53]:
!pip install geocoder

     |████████████████████████████████| 102kB 7.5MB/s 


access info failed

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude, longitude)

read csv

In [63]:
geo=pd.read_csv('Geospatial_Coordinates.csv'); geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


rename the column for merge

In [65]:
geo=geo.rename(columns={'Postal Code':'PostalCode'}); geo.head(1)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353


merge and display

In [66]:
df1=pd.merge(df, geo, how='left', on='PostalCode'); df1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
